In [26]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import numpy as np

In [2]:
import joblib
model = joblib.load('RFR model.pkl')

In [100]:
df = pd.read_excel(".\input\input mixed features.xlsx")
df  = df.iloc[1:2]
df['Namo numeris:'] = df['Namo numeris:'].astype(int)
df = df[["Buto numeris:", "Apsauga:","Namo numeris:","Kambarių sk.:"]]
df

,Buto numeris:,Apsauga:,Namo numeris:,Kambarių sk.:
1,24,NaN,4,3.0


In [101]:
df.dtypes

Buto numeris:     object
Apsauga:          object
Namo numeris:      int32
Kambarių sk.:    float64
dtype: object

In [111]:
Pipe = Pipeline(steps = [
    ("impute", SimpleImputer(strategy = "constant", fill_value=0)),
    ("CustomTransformer", CustomTransformer())
])

In [112]:
Pipe.fit_transform(df)

TypeError: fit_transform() takes 2 positional arguments but 3 were given

In [110]:
from sklearn.base import BaseEstimator, TransformerMixin

class CategoricalInputer(BaseEstimator, TransformerMixin):

    def __init__(self, columns: Optional[List[str]] = None, suffix: str = "_na"):
        self.columns = columns
        self.suffix = suffix

    def fit(self, data: pd.DataFrame) -> "CategoricalInputer":

        self.statistics_ = {
            column: data[column].mode().values[0]
            for column in self.columns or data.columns
        }
        return self


    def transform(self, data: pd.DataFrame) -> pd.DataFrame:

        data = data.copy()
        for column in self.statistics_:
            data.insert(
                data.columns.get_loc(column) + 1,
                column + self.suffix,
                data[column].isna().astype(int),
            )

            data[column].fillna(value=self.statistics_[column], inplace=True)

        return data